# Connect ECG file and csv

In [6]:
import os
import glob

# 指定文件夹路径
folder_path = "/data1/1shared/lijun/data/Huake/Huake-xml"

# 使用 glob 获取文件夹下所有的 XML 文件
input_files = glob.glob(os.path.join(folder_path, "*.xml"))

# 打印所有的 XML 文件路径
for file_path in input_files:
    print(file_path)

/data1/1shared/lijun/data/Huake/Huake-xml/22854661_1_╒┼╚Ё╒ф.xml
/data1/1shared/lijun/data/Huake/Huake-xml/32874777_1_╒┼╜ї┴·.xml
/data1/1shared/lijun/data/Huake/Huake-xml/23122843_1_└ю┤╘╔·.xml
/data1/1shared/lijun/data/Huake/Huake-xml/26804971_1_═ї┴в╙┬.xml
/data1/1shared/lijun/data/Huake/Huake-xml/WHTECG2020060500203_╦я├╧╜и.xml
/data1/1shared/lijun/data/Huake/Huake-xml/27332669_1_┬▐╧у.xml
/data1/1shared/lijun/data/Huake/Huake-xml/WHTECG2019122200430_╜к╗с╡├.xml
/data1/1shared/lijun/data/Huake/Huake-xml/GE20171114234919835_╠я╥л╫┌.xml
/data1/1shared/lijun/data/Huake/Huake-xml/29342220_1_╘м╓╛╞╜.xml
/data1/1shared/lijun/data/Huake/Huake-xml/31426707_1_═ї╕╒.xml
/data1/1shared/lijun/data/Huake/Huake-xml/22075197_1_┤▐ц┬.xml
/data1/1shared/lijun/data/Huake/Huake-xml/FLPtc201304201412010001_┴°╠ь├ў.xml
/data1/1shared/lijun/data/Huake/Huake-xml/28105955_1_╗╞╡┬├ў.xml
/data1/1shared/lijun/data/Huake/Huake-xml/22560356_1_╗╞╣є▒ж.xml
/data1/1shared/lijun/data/Huake/Huake-xml/23984345_1_┬└║г╜н.xml
/data1

In [7]:

import xml.etree.ElementTree as ET
import pandas as pd
import csv

# 输入 XML 文件路径列表
# input_files = ["/data1/1shared/lijun/data/Huake/Huake-xml/1101211606010841369842258_╒┼╧р╠й.xml"]
output_csv_path = "/data1/1shared/lijun/ecg/ECG-EchoReport/preprocess/parsed_patient_study_uid_str.csv"

# 定义函数以解析 XML 并提取 <PatientID> 和 <StudyInstanceUID>，确保输出为字符串格式
def parse_patient_study_info_as_str(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    data = []

    # 遍历 PATIENT 和 STUDY 节点
    for patient in root.findall(".//PATIENT"):
        patient_id = patient.find("PatientID").text if patient.find("PatientID") is not None else ""
        patient_uid = patient.find("PatientUID").text if patient.find("PatientUID") is not None else ""

        # 查找对应的 STUDY 节点
        if patient_uid:
            study_elements = root.findall(f".//STUDY[PatientUID='{patient_uid}']")
            for study in study_elements:
                study_uid = study.find("StudyInstanceUID").text if study.find("StudyInstanceUID") is not None else ""
                if patient_id and study_uid:
                    data.append({"PatientID": str(patient_id), "StudyInstanceUID": str(study_uid)})

    return data

# 解析所有 XML 文件并收集数据
all_data_str = []
for xml_file in input_files:
    all_data_str.extend(parse_patient_study_info_as_str(xml_file))

# 创建 DataFrame 并保存为 CSV 文件
df_str = pd.DataFrame(all_data_str)
df_str.to_csv(output_csv_path, index=False)

df_str

,PatientID,StudyInstanceUID
0,1003400592,22854661_1
1,1006912253,32874777_1
2,1003715748,23122843_1
3,1005050397,26804971_1
4,2001356337,WHTECG2020060500203
...,...,...
5974,1004576661,26291023_1
5975,1009751898,WHTECG2020011301272
5976,1007554334,WHTECG2018082300621
5977,1004923644,GE20161129020658274


In [7]:
import pandas as pd

# 加载两个 CSV 文件
data1 = pd.read_csv('/data1/1shared/lijun/ecg/ECG-EchoReport/preprocess/parsed_patient_study_uid_str.csv')
data2 = pd.read_excel('/data1/1shared/lijun/ecg/ECG-EchoReport/preprocess/Huake_EchoReport.xlsx')


# 通过 'hadm_id' 列将两个数据框合并
merged_data = pd.merge(data1, data2, on='PatientID', how='inner')

# 保存合并后的数据到新的 CSV 文件
merged_data.to_csv('/data1/1shared/lijun/ecg/ECG-EchoReport/preprocess/Huake_ECG_Echo_report.csv', index=False)
merged_data

,PatientID,StudyInstanceUID,ECG,检查类别,性别,检查项目,临床症状,临床诊断,病人来源,描述,...,心尖,室壁运动异常,室壁瘤,EF,三尖瓣,肺动脉瓣,E'(cm/s),A'(cm/s),E/Em,心包积液
0,1003400592,22854661_1,NaN,US,女,"【】彩超-组织多普勒显像(TDI),彩超-心脏+心功能+室壁运动分析",NaN,左下肺肿块,病房,1.右室流出道不增宽（31mm），主动脉根部不增宽（28mm），升主动脉近端增宽（38mm）...,...,NaN,未见节段性室壁运动异常,无,66%,三尖瓣口右房侧收缩期可见重度返流血流信号及湍流频谱:PFV：3.4m/s，PG：45mmHg。,,,,,
1,1006912253,32874777_1,NaN,XNUS,男,【】彩超-心脏+心功能+室壁运动分析,NaN,NaN,病房,1.左心室增大（6.5cm），左房增大（5.0*5.3*6.2cm），右房增大（5.2*6....,...,NaN,左室弥漫性室壁运动减低,无,11%,未见明显异常，右房侧收缩期见中-重度返流信号，PFV=475cm/s，PG=90mmHg。,未见明显异常，肺动脉瓣舒张期右室侧可见轻度返流信号,2,2,45,
2,1006912253,32874777_1,NaN,XNUS,男,【】彩超-心脏+心功能+室壁运动分析,NaN,冠心病 PCI术后 心功能IV级,病房,1.左心室增大（6.5cm），左房增大（5.0*5.6*6.6cm），右房增大（5.1*6....,...,NaN,左室弥漫性室壁运动减低,无,13%,未见明显异常，右房侧收缩期见中-重度返流信号，PFV=475cm/s，PG=90mmHg。,未见明显异常，肺动脉瓣舒张期右室侧可见轻度返流信号,2,8,27,心包脏壁层分离，最大液性暗区为左室侧壁0.5cm。
3,1003715748,23122843_1,NaN,US,女,"【】彩超-组织多普勒显像(TDI),彩超-心脏+心功能+室壁运动分析",先天性心脏病 房间隔缺损 肺动脉高压 心功能Ⅳ级,先天性心脏病 房间隔缺损 肺动脉高压 心功能Ⅳ级,病房,1.右室流出道增宽（38mm），主动脉根部不增宽（23mm），主动脉瓣回声正常，开放可。肺动...,...,NaN,未见节段性室壁运动异常,无,66%,三尖瓣口右房侧收缩期可见重度返流血流信号及湍流频谱，PFV：4.7m/s，PG：88mmHg。,,,,,
4,1003715748,23122843_1,NaN,US,女,"【】彩超-组织多普勒显像(TDI),彩超-心脏+心功能+室壁运动分析",先天性心脏病 房间隔缺损 肺动脉高压 心功能Ⅳ级,先天性心脏病 房间隔缺损 肺动脉高压 心功能Ⅳ级,病房,1.右室流出道稍增宽（34mm），主动脉根部不增宽（23mm），升主动脉近端不增宽（23mm...,...,NaN,未见节段性室壁运动异常,无,57%,三尖瓣口右房侧收缩期可见轻度返流血流信号及湍流频谱:PFV：3.0m/s，PG：36mmHg。,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5084,1004806656,GE20150828125442220,"+,右室",US,女,"【】彩超-组织多普勒显像(TDI),彩超-心脏+心功能+室壁运动分析",NaN,慢性阻塞性肺疾病 急性发作期 慢性肺源性心脏病 Ⅱ型呼吸功能衰竭,病房,1.右室流出道不增宽（29mm），主动脉根部不增宽（27mm），升主动脉近端不增宽（28mm...,...,NaN,未见节段性室壁运动异常,无,70%,三尖瓣口右房侧收缩期可见轻度返流血流信号及湍流频谱:PFV：3.6m/s，PG：53mmHg。,,,,,
5085,1003613936,29212887_1,NaN,XNUS,男,【】彩超-心脏+心功能+室壁运动分析,看心包积液,全心扩大原因待查：扩张型心肌病？缺血型心肌病？,病房,1.左心室增大（5.6cm），左房增大（5.1*4.7*5.8cm），右房增大（5.3*6....,...,NaN,左室弥漫性室壁运动减低,无,47%,未见明显异常，右房侧收缩期见重度返流信号，PFV=275cm/s，PG=30mmHg。,未见明显异常，肺动脉瓣舒张期右室侧未见明显返流信号,8,4,11,心包脏壁层分离，最大液性暗区为左室后壁0.4cm，左室侧壁0.6cm，右房侧壁0.8cm。
5086,1004923644,GE20161129020658274,NaN,XNUS,男,【】彩超-心脏+心功能+室壁运动分析,NaN,慢性阻塞性肺疾病急性发作期,病房,患者图像质量欠佳：\n\n 1.左心室不大（4.8cm），左房增大（5.1*6.0*7...,...,NaN,左室未见明显节段性室壁运动异常,无,49%,未见明显异常，右房侧收缩期见中度返流信号，PFV=329cm/s，PG=43mmHg。,未见明显异常，肺动脉瓣舒张期右室侧见轻度返流信号,6,4,14,心包脏壁层分离，最大液性暗区为左室侧壁0.4cm。
5087,1004923644,GE20161129020658274,NaN,XNUS,男,【】彩超-心脏+心功能+室壁运动分析,因胸闷气喘由呼吸科转入我科。,慢性阻塞性肺疾病急性发作期,病房,患者图像质量欠佳：\n\n 1.左心室不大（4.8cm），左房增大（4.6*5.4*6...,...,NaN,左室未见明显节段性室壁运动异常,无,51%,未见明显异常，右房侧收缩期见轻-中度返流信号，PFV=249cm/s，PG=25mmHg。,未见明显异常，肺动脉瓣舒张期右室侧见轻度返流信号,3,4,15,


# Transform the ECG file from xlsx to npy

In [8]:
import os
import numpy as np
import pandas as pd

# Define the directory containing the Excel files and the output directory for .npy files
input_directory = '/data1/1shared/lijun/data/Huake/Huake-xlsx'
output_directory = '/data1/1shared/lijun/data/Huake/Huake-npy'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Define the target shape
num_leads = 12
target_samples = 5000

# Loop over all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(input_directory, filename)
        
        # Load the Excel file
        excel_data = pd.read_excel(file_path, sheet_name='Sheet1')
        
        # Extract numerical data starting from row 4
        ecg_signals = excel_data.iloc[3:].to_numpy(dtype=float)
        
        # Trim/pad each lead to match (12, 5000) format
        ecg_signals_trimmed = []
        for i in range(num_leads):
            lead_data = ecg_signals[:, i]
            if len(lead_data) >= target_samples:
                ecg_signals_trimmed.append(lead_data[:target_samples])
            else:
                # Pad with zeros if fewer samples
                padded_data = np.pad(lead_data, (0, target_samples - len(lead_data)), 'constant')
                ecg_signals_trimmed.append(padded_data)
        
        # Convert to numpy array of shape (12, 5000)
        ecg_array = np.array(ecg_signals_trimmed)
        
        # Save the array as .npy with the same base name as the Excel file
        output_filename = os.path.splitext(filename)[0] + '.npy'
        output_path = os.path.join(output_directory, output_filename)
        np.save(output_path, ecg_array)
        
        print(f"Processed and saved: {output_filename}")


Processed and saved: 28639146_1ECG.npy
Processed and saved: 29910104_1ECG.npy
Processed and saved: WHTECG2019100800941ECG.npy
Processed and saved: 33318159_1ECG.npy
Processed and saved: 24580645_1ECG.npy
Processed and saved: 33221083_1ECG.npy
Processed and saved: 8488754_1ECG.npy
Processed and saved: WHTECG2019030401072ECG.npy
Processed and saved: 28464237_1ECG.npy
Processed and saved: WHTJYY2017101300163ECG.npy
Processed and saved: WHTECG2018102000677ECG.npy
Processed and saved: 31249410_1ECG.npy
Processed and saved: 29743158_1ECG.npy
Processed and saved: WHTECG2020010100301ECG.npy
Processed and saved: WHTECG2018091201206ECG.npy
Processed and saved: WHTECG2019040301297ECG.npy
Processed and saved: 25725362_1ECG.npy
Processed and saved: WHTECG2019090401731ECG.npy
Processed and saved: WHTECG2018120300916ECG.npy
Processed and saved: 28863023_1ECG.npy
Processed and saved: GE20160707114548353ECG.npy
Processed and saved: 29322756_1ECG.npy
Processed and saved: WHTECG2019072201404ECG.npy
Proce